In [ ]:
!pip install contractions
!pip install transformers
!pip install rank-bm25
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 12.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Read and Split the given Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


###Preprocessing

In [ ]:
# read data from the json files
train_df = pd.read_json("train.jsonl", lines = True)
validation_df = pd.read_json("validation.jsonl", lines = True)

print("train data shape:",train_df.shape)
print("validation data shape:",validation_df.shape)

# Merge the complete data to divide the data train, test and validation

total_df = pd.concat([train_df, validation_df], ignore_index = True)
print("total data shape:",total_df.shape)

train data shape: (3200, 14)
validation data shape: (800, 14)


In [ ]:
total_df = total_df[["uuid", "postText", "targetParagraphs", "spoiler", "tags","targetTitle","targetDescription","spoilerPositions","provenance"]]
total_df.rename(columns={"postText":"clickbait", "targetParagraphs":"document", "tags":"type"}, inplace = True)

<ipython-input-7-2dc82af6a2d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.rename(columns={"postText":"clickbait", "targetParagraphs":"document", "tags":"type"}, inplace = True)


### Preprocessing and BM25 Implementation

In [ ]:
# Function to Make values in Spoiler_Type(list to string)
def list_to_string(spoiler_type):
  if spoiler_type[0] == "phrase":
    return 0
  elif spoiler_type[0] == "passage":
    return 1
  elif spoiler_type[0] == "multi":
    return 2

total_df["type"] = total_df["type"].apply(list_to_string)

total_df_bkp = total_df.copy()
total_df = total_df[total_df['type']==1]
total_df.reset_index(drop=True,inplace=True)

<ipython-input-8-5d26f8770338>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df["type"] = total_df["type"].apply(list_to_string)


In [ ]:
def append_text(x):
  if(x['targetDescription']!=None):
    x['document'].append(x['targetTitle'])
  return x['document']

In [ ]:
total_df['document'] = total_df.apply(append_text,axis=1)

In [ ]:
def extract_human_spoiler(x):
  return x['provenance']['humanSpoiler']

In [ ]:
def tokenize_corpus(x):
  return [sen.split(" ") for sen in x]

In [ ]:
total_df['tokenized_doc'] = total_df['document'].apply(tokenize_corpus)

In [ ]:
from rank_bm25 import BM25Okapi
total_df['tokenized_doc_bm25'] = total_df['tokenized_doc'].apply(lambda x: BM25Okapi(x))

In [ ]:
total_df.head(2)

,uuid,clickbait,document,spoiler,type,targetTitle,targetDescription,spoilerPositions,provenance,tokenized_doc,tokenized_doc_bm25,tokenized_clickbait,dm25_scores,output_sentence,bm25_scores
0,0af11f6b-c889-4520-9372-66ba25cb7657,"[Wes Welker Wanted Dinner With Tom Brady, But ...",[It’ll be just like old times this weekend for...,[how about that morning we go throw?],1,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"[[[3, 151], [3, 186]]]","{'source': 'anonymized', 'humanSpoiler': 'They...","[[It’ll, be, just, like, old, times, this, wee...",<rank_bm25.BM25Okapi object at 0x7fadeea663d0>,"[[Wes, Welker, Wanted, Dinner, With, Tom, Brad...","[2.6778664701405854, 0.3312983287185408, 0.483...","[Wes Welker Wanted Dinner With Tom Brady, But ...","[2.6778664701405854, 0.3312983287185408, 0.483..."
1,12e3034e-a98f-4cd2-8773-3f7974161c45,[What happens if your new AirPods get lost or ...,[One of the biggest surprise announcements at ...,[Apple says that if AirPods are lost or stolen...,1,Here's what happens if your Apple AirPods get ...,One of the first questions we had about Apple'...,"[[[4, 0], [4, 110]]]","{'source': 'anonymized', 'humanSpoiler': 'No',...","[[One, of, the, biggest, surprise, announcemen...",<rank_bm25.BM25Okapi object at 0x7fadeea66370>,"[[What, happens, if, your, new, AirPods, get, ...","[0.0, 2.0895428421687177, 3.821048370397315, 0...",[Here's what happens if your Apple AirPods get...,"[0.0, 2.0895428421687177, 3.821048370397315, 0..."


In [ ]:
total_df['tokenized_clickbait'] = total_df['clickbait'].apply(tokenize_corpus)

In [ ]:
def get_bm25_scores(x):
  scores = x['tokenized_doc_bm25'].get_scores(x['tokenized_clickbait'][0])
  # print(scores)
  return scores

In [ ]:
total_df['bm25_scores'] = total_df.apply(get_bm25_scores,axis=1)

In [ ]:
def get_spoiler(x):
  output_spoiler = x['tokenized_doc_bm25'].get_top_n(x['tokenized_clickbait'][0],x['document'],n=1)
  # print(output_spoiler)
  return output_spoiler

In [ ]:
total_df['output_sentence'] = total_df.apply(get_spoiler,axis=1)

### Evaluation metric

In [1]:
# BLEU

import evaluate
bleu = evaluate.load("bleu")
from rank_bm25 import BM25Okapi
count = 0
results = 0

for index, row in total_df.iterrows():

  corpus = row["document"]

  tokenized_corpus = [doc.split(" ") for doc in corpus]

  bm25 = BM25Okapi(tokenized_corpus)

  query = row["clickbait"][0]
  tokenized_query = query.split(" ")

  doc_scores = bm25.get_scores(tokenized_query)

  doc_scores = list(doc_scores)
  sentence_index = doc_scores.index(max(doc_scores))

  predicted_sentence = [row["document"][sentence_index]]


  # Bleu score
  predictions = predicted_sentence
  references = [[row['spoiler']]]
  results += bleu.compute(predictions=predictions, references=references).get("bleu")
  count += 1

  if count == 100:
    break

bleu_score = results/count
print("BLEU Score: ", bleu_score)


ModuleNotFoundError: ignored

In [ ]:
# METEOR

import evaluate
meteor = evaluate.load('meteor')
from rank_bm25 import BM25Okapi
count = 0
results = 0

for index, row in total_df.iterrows():

  corpus = row["document"]

  tokenized_corpus = [doc.split(" ") for doc in corpus]

  bm25 = BM25Okapi(tokenized_corpus)

  query = row["clickbait"][0]
  tokenized_query = query.split(" ")

  doc_scores = bm25.get_scores(tokenized_query)

  doc_scores = list(doc_scores)
  sentence_index = doc_scores.index(max(doc_scores))

  predicted_sentence = [row["document"][sentence_index]]


  # Meteor score
  predictions = [f'"{predicted_sentence}"']
  spoiler = row['spoiler']
  references = [
                  [f'"{spoiler}"'], 
    ]
  print("1. predictions: ", predicted_sentence)
  print("2. spoilers: ", spoiler)
  print("\n")
  results += meteor.compute(predictions=predictions, references=references).get("meteor")
  count += 1


  if count == 10:
    break

meteor_score = results/count
print("METEOR Score: ", meteor_score)


In [ ]:
import evaluate
from rank_bm25 import BM25Okapi
bertscore = load("bertscore")

results = 0
total_precision = 0
total_recall = 0
total_f1 = 0

count = 0

for index, row in total_df.iterrows():

  corpus = row["document"]

  tokenized_corpus = [doc.split(" ") for doc in corpus]

  bm25 = BM25Okapi(tokenized_corpus)

  query = row["clickbait"][0]
  tokenized_query = query.split(" ")

  doc_scores = bm25.get_scores(tokenized_query)

  doc_scores = list(doc_scores)
  sentence_index = doc_scores.index(max(doc_scores))

  predicted_sentence = [row["document"][sentence_index]]


  # BERT score
  predictions = [f'"{predicted_sentence}"']
  spoiler = row["spoiler"]
  references = [
                  f'"{spoiler}"'
    ]

  print("1. predictions: ", predicted_sentence)
  print("2. spoilers: ", spoiler)
  print("\n")

  results = bertscore.compute(predictions=predictions, references=references, lang="en")
  total_precision += results.get("precision")[0]
  total_recall += results.get("recall")[0]
  total_f1 += results.get("f1")[0]
  print(count)
  count += 1

  if count == 10:
    break

avg_precision = total_precision/count
avg_recall = total_recall/count
avg_f1 = total_f1/count

print("Avg. Precision value: ", avg_precision)
print("Avg. Recall value: ", avg_recall)
print("Avg. F1 value: ", avg_f1)



1. predictions:  ['Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had A Better Idea']
2. spoilers:  ['how about that morning we go throw?']


0
1. predictions:  ["Here's what happens if your Apple AirPods get lost or stolen"]
2. spoilers:  ["Apple says that if AirPods are lost or stolen, you'll have to buy new ones, just like any other Apple product."]


1
1. predictions:  ['The Reason Why Gabor Kiraly Wears THOSE Trackie Bottoms']
2. spoilers:  ['"The more good games I had in them, the more I got used to them.']


2
1. predictions:  ['"It was cold and very foggy, the temperature was around -10 degrees Celsius," said Hänninen. "When the clouds began to break, there were rainbow colours in the sky and a halo spanning 360 degrees! It was worth taking a picture or two."']
2. spoilers:  ['rainbow colours in the sky and a halo spanning 360 degrees']


3
1. predictions:  ['Should I Drink Red Wine?']
2. spoilers:  ['Red wine is clearly the drink of choice if you are doing light to 